### Importing required Libraries

In [8]:
import os as os
import pandas as pd

In [9]:
os.chdir('D:/Readings/Analytics Vidhya/Loan Prediction Hackathon')

#### Importing Dataset

In [10]:
df = pd.read_csv("train_ctrUa4K.csv")

In [11]:
df.columns

Index(['Loan_ID', 'Gender', 'Married', 'Dependents', 'Education',
       'Self_Employed', 'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Property_Area', 'Loan_Status'],
      dtype='object')

In [12]:
df.dtypes

Loan_ID               object
Gender                object
Married               object
Dependents            object
Education             object
Self_Employed         object
ApplicantIncome        int64
CoapplicantIncome    float64
LoanAmount           float64
Loan_Amount_Term     float64
Credit_History       float64
Property_Area         object
Loan_Status           object
dtype: object

#### Getting all categorical variables

In [13]:
df_cat = pd.DataFrame(data = df.dtypes, columns = ['a']).reset_index()

In [14]:
df_cat.head()

,index,a
0,Loan_ID,object
1,Gender,object
2,Married,object
3,Dependents,object
4,Education,object


In [15]:
cat_var = list(df_cat['index'].loc[df_cat['a'] == 'object'])

In [16]:
cat_var

['Loan_ID',
 'Gender',
 'Married',
 'Dependents',
 'Education',
 'Self_Employed',
 'Property_Area',
 'Loan_Status']

In [17]:
df_cat = df[cat_var]

In [18]:
df_cat.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,Urban,Y
4,LP001008,Male,No,0,Graduate,No,Urban,Y


#### Removing records with at least one null value in a row

In [19]:
df_cat_v1 = df_cat.dropna()

In [20]:
df_cat_v1.shape

(554, 8)

Let us split this list into two parts

In [21]:
cat_var1 = ('Gender', 'Married', 'Dependents', 'Education', 'Self_Employed', 'Property_Area')

In [22]:
cat_var2 = ('Gender', 'Married', 'Dependents', 'Education', 'Self_Employed', 'Property_Area')

#### Let us jump to Chi-Square test. To start with let us import the required libraries

In [23]:
from itertools import product
import numpy as np
import scipy.stats as ss

Creating all possible combinations between the above two variables list

In [24]:
cat_var_prod = list(product(cat_var1,cat_var2, repeat = 1))

In [25]:
cat_var_prod.drop_duplicates()

AttributeError: 'list' object has no attribute 'drop_duplicates'

So we have in all 12 ombinations. Now just imagine if you were to really check association between each of these variables,
you would have to run chi-square test 12 times. So here I will help you with the code that will reduce the execution times from
12 to 1. Let us see how we can do this.

Creating an empty variable and picking only the p value from the output of Chi-Square test

In [18]:
result = []

In [19]:
for i in cat_var_prod:
    if i[0] != i[1]:
        result.append((i[0], i[1],list(ss.chi2_contingency(pd.crosstab(df_cat_v1[i[0]], df_cat_v1[i[1]])))[1]))

In [60]:
result

[('Gender', 'Married', 3.195919822839657e-17),
 ('Gender', 'Dependents', 2.578960792620006e-05),
 ('Gender', 'Education', 0.2988070774266799),
 ('Gender', 'Self_Employed', 0.8367073961290672),
 ('Gender', 'Property_Area', 0.05166338663302401),
 ('Married', 'Gender', 3.195919822839657e-17),
 ('Married', 'Dependents', 4.622117038501888e-18),
 ('Married', 'Education', 0.9780364597922157),
 ('Married', 'Self_Employed', 0.9667467939322089),
 ('Married', 'Property_Area', 0.885151312411991),
 ('Dependents', 'Gender', 2.578960792620006e-05),
 ('Dependents', 'Married', 4.622117038501888e-18),
 ('Dependents', 'Education', 0.3262645381335668),
 ('Dependents', 'Self_Employed', 0.10758819405474623),
 ('Dependents', 'Property_Area', 0.25342662767163165),
 ('Education', 'Gender', 0.2988070774266799),
 ('Education', 'Married', 0.9780364597922157),
 ('Education', 'Dependents', 0.3262645381335668),
 ('Education', 'Self_Employed', 0.9115980403762441),
 ('Education', 'Property_Area', 0.3918935252051685),


Now the idea here is to create a crosstab similar to what we get from df.corr() function

In [21]:
chi_test_output = pd.DataFrame(result, columns = ['var1', 'var2', 'coeff'])

Using pivot function to convert the above DataFrame into a crosstab

In [22]:
chi_test_output.pivot(index='var1', columns='var2', values='coeff')

var2,Dependents,Education,Gender,Married,Property_Area,Self_Employed
var1,,,,,,
Dependents,NaN,0.326265,2.578961e-05,4.622117e-18,0.253427,0.107588
Education,3.262645e-01,NaN,2.988071e-01,9.780365e-01,0.391894,0.911598
Gender,2.578961e-05,0.298807,NaN,3.195920e-17,0.051663,0.836707
Married,4.622117e-18,0.978036,3.195920e-17,NaN,0.885151,0.966747
Property_Area,2.534266e-01,0.391894,5.166339e-02,8.851513e-01,NaN,0.704156
Self_Employed,1.075882e-01,0.911598,8.367074e-01,9.667468e-01,0.704156,NaN


##### There exists a relationship between two variables if p value < 0.05. So from the above table we can say that there is definetly some association between Dependents and Gender, Dependents and Married, Married and Gender. For rest of the pairs there exists no relationship 

## Post Hoc Testing

In [28]:
for i in cat_var_prod:
    if i[0] != i[1]:
        result1.append((i[0], i[1],list(ss.chi2_contingency(pd.crosstab(df_cat_v1[i[0]], df_cat_v1[i[1]])))[3]))

In [59]:
list(map(itemgetter(2), result1))

[array([[ 35.00361011,  65.99638989],
        [156.99638989, 296.00361011]]),
 array([[ 58.15703971,  17.31949458,  17.13718412,   8.38628159],
        [260.84296029,  77.68050542,  76.86281588,  37.61371841]]),
 array([[ 78.57581227,  22.42418773],
        [352.42418773, 100.57581227]]),
 array([[ 87.14440433,  13.85559567],
        [390.85559567,  62.14440433]]),
 array([[ 29.7166065 ,  38.46750903,  32.81588448],
        [133.2833935 , 172.53249097, 147.18411552]]),
 array([[ 35.00361011, 156.99638989],
        [ 65.99638989, 296.00361011]]),
 array([[110.55595668,  32.92418773,  32.57761733,  15.94223827],
        [208.44404332,  62.07581227,  61.42238267,  30.05776173]]),
 array([[149.37184116,  42.62815884],
        [281.62815884,  80.37184116]]),
 array([[165.66064982,  26.33935018],
        [312.33935018,  49.66064982]]),
 array([[ 56.49097473,  73.12635379,  62.38267148],
        [106.50902527, 137.87364621, 117.61732852]]),
 array([[ 58.15703971, 260.84296029],
        [ 17.3